In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 20230929
# 下面的代码不用了 直接使用git clone https://huggingface.co/THUDM/chatglm2-6b-32k 就能下载模型bin和各类附属的小文件了

In [ ]:
import transformers
from transformers import AutoModel,AutoTokenizer
model = AutoModel.from_pretrained("THUDM/chatglm2-6b",trust_remote_code=True,device_map="auto")
model.save_pretrained("local_chatglm2")

In [ ]:
tokenzier= AutoTokenizer.from_pretrained("THUDM/chatglm2-6b",trust_remote_code=True)
tokenzier.save_pretrained("local_chatglm2")

# 还有一个《先SFT再DPO_0829》 在hughug账户
# https://www.kaggle.com/code/hughug/sft-dpo-0829#%E6%99%AE%E9%80%9Alora-%E5%BD%93ref_model=None%E6%97%B6-lora-target-modules=%5Bdefault%E3%80%82%E3%80%82%E3%80%82%5D%E5%B1%85%E7%84%B6%E4%BC%9A%E6%B7%B7%E5%85%A5%E5%A5%87%E6%80%AA%E7%9A%84default-module-%E6%88%91%E5%8F%AA%E5%A5%BD%E5%9C%A8%E5%87%BD%E6%95%B0%E4%B8%AD%E5%BC%BA%E5%88%B6%E5%88%A0%E9%99%A4

# xuming的colab演示 dpo
https://colab.research.google.com/drive/1kMIe3pTec2snQvLBA00Br8ND1_zwy3Gr?usp=sharing#scrollTo=Xmltp4ILwSfu

In [ ]:
%cd /kaggle/working

In [ ]:
rm -rf MedicalGPT

# 拉取特定分支 git clone -b 

In [ ]:
%cd /kaggle/working
!git clone -b medGPT_0828 https://github.com/valkryhx/MedicalGPT

# 进入项目检查分支是否拉取正确 git status

In [ ]:
%cd MedicalGPT
!git status

In [ ]:
%cd MedicalGPT
!git pull --all --force
!pip install -r requirements.txt

# 按照colab演示 开始

In [ ]:
%ls ./data/reward/

In [ ]:
#cat ./data/reward/test.json

!python dpo_training.py \
    --model_type bloom \
    --model_name_or_path merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 50 \
    --max_source_length 128 \
    --max_target_length 128 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

# peft==0.5.0 则accelerate必须为0.21.0 不然会报错
# [ImportError: cannot import name 'is_npu_available' from 'accelerate.utils]
# https://github.com/eosphoros-ai/DB-GPT-Hub/issues/42

In [ ]:
# requirements.txt中已经是了
#!pip install accelerate==0.21

# 下面是没有使用qlora的 能跑！

In [ ]:
!git pull --all --force
!python dpo_training.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b  \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 40 \
    --max_source_length 128 \
    --max_target_length 128 \
    --output_dir outputs-dpo-0924 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

In [ ]:
# for qlora
!pip install bitsandbytes==0.39.0

# 改成 自己的dpo_train_2.py

In [ ]:
#!pip install peft==0.5.0

# 不要用 dpo_train_2.py  这个无法正常运行
# 可以用dpo_for_peftmodel.py
# 可以用dpo_peftmode_my_style_0830.py 这是我自己的用法 能用luzi.json先传一部分参数

In [ ]:
!git pull --all --force
!python dpo_train_2.py \
    --qlora False \
    --learning_rate 2e-5 \
    --model_type chatglm \
    --model_name_or_path /kaggle/working/MedicalGPT/outputs-dpo-0924/checkpoint-80  \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 40 \
    --max_source_length 128 \
    --max_target_length 128 \
    --output_dir outputs-dpo-0924-v2 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

# 使用AutoPeftModelForCausalLM的写法的dpo_peftmodel_my_style_0830.py 可以run  
# 先不使用qlora  设置qlora=False   ,use_ref_model=True(默认)  会爆显存oom  
# 先不使用qlora 设置qlora=False  ,use_ref_model=False ,load_in_4bit=True 会报ValueError: Target modules [] not found in the base model. Please check the target modules and try again.  这可能是因为加载的model本身就是peftmodel了 不能再量化了
# qlora=False ,use_ref_model=False ,load_in_4bit=False 爆显存 差一点点就行
# <font color=red>qlora=True ,use_ref_model=False ,load_in_4bit=True 可以运行 </font>

In [ ]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_peftmodel_my_style_0830.py \
    --model_type chatglm \
    --model_name_or_path /kaggle/working/MedicalGPT/outputs-dpo-0924 \
    --tokenizer_name_or_path THUDM/chatglm2-6b \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --use_ref_model False \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-0924-v2-no-qlora \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --train_args_json luzi.json \
    --compute_dtype fp16 

# 使用最新的将basemodel和adapter分开加载的dpo train new 脚本

# 在单张A800上可以使用
# CUDA_VISIBLE_DEVICES=0 python dpo_my_style_0926_new.py

# 测试不使用--resume_from_checkpoint的效果  可以run

In [ ]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_my_style_0926_new.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --use_ref_model True \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-0924-v2-no-qlora \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --train_args_json luzi.json \
    --compute_dtype fp16 

# 使用最新的将basemodel和adapter分开加载的dpo train new 脚本

# 在单张A800上可以使用
# CUDA_VISIBLE_DEVICES=0 python dpo_my_style_0926_new.py

# 测试开启--resume_from_checkpoint的效果  可以run

In [ ]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_my_style_0926_new.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b \
    --resume_from_checkpoint  /kaggle/working/MedicalGPT/outputs-dpo-0924 \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --use_ref_model True \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-0924-v2-no-qlora \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --train_args_json luzi.json \
    --compute_dtype fp16 

In [ ]:
!git pull --force --all

# 用yunguan_langchain_0926里面的langchain格式样例跑 设置--max_source_length 1024 \ --max_target_length 1200 \ --gradient_accumulation_steps 4 居然可以没有oom，显存占用14.7G

# <font color=red>非继续训练  这里使用本地的chatglm2文件 也能运行qlora  --use_ref_model False \
# 注意bs=1 gradient_accumulation_steps=4   GPU0=14.5G  GPU1=9.7G</font>

In [ ]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_my_style_0926_new.py \
    --model_type chatglm \
    --model_name_or_path /kaggle/working/local_chatglm2 \
    --train_file_dir ./data/yunguan_langchain_0926 \
    --validation_file_dir ./data/yunguan_langchain_0926 \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --use_ref_model False \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 1024 \
    --max_target_length 1200 \
    --output_dir output_dpo_0926_qlora \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --compute_dtype fp16 \
    --fp16 True \
    --device_map auto \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --train_args_json luzi.json

# <font color=red>继续训练  这里使用本地的chatglm2文件 也能运行qlora 测试  use_ref_model True
# 注意bs=1 gradient_accumulation_steps=4   GPU0=14.5G  GPU1=9.7G</font>

In [ ]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_my_style_0926_new.py \
    --model_type chatglm \
    --model_name_or_path /kaggle/working/local_chatglm2 \
    --resume_from_checkpoint /kaggle/working/MedicalGPT/output_dpo_0926_qlora/checkpoint-40 \
    --train_file_dir ./data/yunguan_langchain_0926 \
    --validation_file_dir ./data/yunguan_langchain_0926 \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --use_ref_model False \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples -1 \
    --max_eval_samples 20 \
    --num_train_epochs 3 \
    --eval_steps 10 \
    --save_steps 20 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 1024 \
    --max_target_length 1200 \
    --output_dir output_dpo_0926_qlora_continually \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --compute_dtype fp16 \
    --fp16 True \
    --device_map auto \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --train_args_json luzi.json

In [ ]:
#20230929 测试chatglm2-6b-32k
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_my_style_0926_new.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b-32k \
    --train_file_dir ./data/yunguan_langchain_0926 \
    --validation_file_dir ./data/yunguan_langchain_0926 \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --use_ref_model False \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 1024 \
    --max_target_length 1200 \
    --output_dir output_dpo_0929_qlora_32k \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --compute_dtype fp16 \
    --fp16 True \
    --device_map auto \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --train_args_json luzi.json

# 为chatglm系列实现的cpu推理llama.cpp 类似的chatglm.cpp   https://github.com/li-plus/chatglm.cpp  可以试试

# GGML
https://huggingface.co/TheBloke/Llama-2-70B-Chat-GGML
https://github.com/ggerganov/ggml

# Llama的GGML  目前最新版的已不支持ggml格式 开始支持gguf格式
https://huggingface.co/TheBloke/Llama-2-70B-Chat-GGML

# GGUF 是GGML团队新开发的格式 
https://huggingface.co/TheBloke/CodeLlama-7B-GGUF

# 试试chatglm.cpp项目 的cpu加速 （gpu加速在后面）

In [ ]:
# 使用Python binding的方式安装  
!CMAKE_ARGS="-DGGML_OPENBLAS=ON" pip install -U chatglm-cpp

In [ ]:
# 下载项目打到本地  主要是要使用其 convert.py 文件来量化模型
%cd /kaggle/working/
!git clone --recursive https://github.com/li-plus/chatglm.cpp.git 

In [ ]:
#ls chatglm.cpp/
%cd chatglm.cpp/

In [ ]:
!python3 chatglm_cpp/convert.py \
-i THUDM/chatglm2-6b \
-l  /kaggle/working/MedicalGPT/output_dpo_0926_qlora_continually/checkpoint-40  \
-t q4_0 \
-o chatglm-ggml.bin

In [ ]:
pwd

In [ ]:
#测试 stream chat   
# CPU 1.9G 但是  kaggle上似乎是for循环还是啥 卡住

In [ ]:
!python3 examples/cli_chat.py -m /kaggle/working/chatglm.cpp/chatglm-ggml.bin -i

In [ ]:
# 测试chat # 不能在chatglm.cpp目录中测
%cd /kaggle/working  
import chatglm_cpp
pipeline = chatglm_cpp.Pipeline("/kaggle/working/chatglm.cpp/chatglm-ggml.bin")


In [ ]:
import time
t_start = time.time()
res = pipeline.chat(["介绍一下中秋节的来历"])
t_end = time.time()
print(res)
t_cost = t_end - t_start
print(f"t_cost={t_cost} s")
print(f"chars_per_second={len(res)/(t_cost)} c/s")

In [ ]:
关于 threads 的含义 是推理时使用的cpu核数  https://github.com/li-plus/chatglm.cpp/issues/62
    examples/web_demo中有设置generation_dict 里面有设置threads
    https://github.com/li-plus/chatglm.cpp/blob/main/examples/web_demo.py#L38

# <font color=red>明天对比下ggml的gpu和 ggml的cpu 以及原始的推理速度</red>

# 加速后GPU推理 速度达到70char/s-80char/s , 50 tokens/s ,一个 token可以表示1-3个汉字 大概。

In [ ]:
# 使用Python binding的方式安装   gpu加速 开启 CMAKE_ARGS="-DGGML_CUBLAS=ON
!CMAKE_ARGS="-DGGML_CUBLAS=ON" pip install -U chatglm-cpp

In [ ]:
import os,torch
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
torch.cuda.device_count() # result is1

In [ ]:
# 测试chat # 不能在chatglm.cpp目录中测
%cd /kaggle/working  
import chatglm_cpp
pipeline = chatglm_cpp.Pipeline("/kaggle/working/chatglm.cpp/chatglm-ggml.bin")


In [ ]:
from transformers import AutoTokenizer
tokenizer= AutoTokenizer.from_pretrained("THUDM/chatglm2-6b",trust_remote_code=True)

In [ ]:
import time
t_start = time.time()
#res = pipeline.chat(["介绍一下中秋节的来历"])
res = pipeline.chat(["讲一讲 圆周率如何根据级数计算"])
t_end = time.time()
print(res)
t_cost = t_end - t_start
print(f"t_cost={t_cost} s")
print(f"chars_per_second={len(res)/(t_cost)} chars/s")
print(f"tokens_per_second={len(tokenizer.encode(res))/(t_cost)} tks/s")

# 普通推理速度测试

In [ ]:
from transformers import AutoTokenizer, AutoModel
import time
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True,device_map='auto')
model = model.eval()


# 下面的普通推理 非量化 占了两块卡 每块显存6-7G
# t_cost=20.126879692077637 s
# chars_per_second=25.0411393972005 chars/s
# tokens_per_second=17.588419338509876 tks/s

In [ ]:
t_start = time.time()
res, history = model.chat(tokenizer, "讲一讲 圆周率如何根据级数计算", history=[])
t_end = time.time()
print(res)
t_cost = t_end - t_start
print(f"t_cost={t_cost} s")
print(f"chars_per_second={len(res)/(t_cost)} chars/s")
print(f"tokens_per_second={len(tokenizer.encode(res))/(t_cost)} tks/s")


# 使用 bitsandbytes 的NF4 量化 2gpu 每块显卡3-3.5G 
# t_cost=25.050681114196777 s
# chars_per_second=23.432496598558913 chars/s
# tokens_per_second=17.44463545753023 tks/s

In [ ]:
# 使用 bitsandbytes 的NF4 量化

!pip install accelerate==0.21.0
!pip install bitsandbytes==0.41.0

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModel
import time
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_quant_type='nf4',
                                  bnb_4bit_use_double_quant=True,
                                  bnb_4bit_compute_dtype=torch.float16)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True,device_map='auto',quantization_config=q_config)
model = model.eval()

In [ ]:
t_start = time.time()
res, history = model.chat(tokenizer, "讲一讲 圆周率如何根据级数计算", history=[])
t_end = time.time()
print(res)
t_cost = t_end - t_start
print(f"t_cost={t_cost} s")
print(f"chars_per_second={len(res)/(t_cost)} chars/s")
print(f"tokens_per_second={len(tokenizer.encode(res))/(t_cost)} tks/s")

# 使用 bitsandbytes 的NF4 量化 1 gpu 显卡5.5G 
# t_cost=13.949055194854736 s
# chars_per_second=27.241988413678886 chars/s
# tokens_per_second=18.782634116799652 tks/s

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModel
import time
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_quant_type='nf4',
                                  bnb_4bit_use_double_quant=True,
                                  bnb_4bit_compute_dtype=torch.float16)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True,device_map={"":1},quantization_config=q_config)
model = model.eval()

In [ ]:
t_start = time.time()
res, history = model.chat(tokenizer, "讲一讲 圆周率如何根据级数计算", history=[])
t_end = time.time()
print(res)
t_cost = t_end - t_start
print(f"t_cost={t_cost} s")
print(f"chars_per_second={len(res)/(t_cost)} chars/s")
print(f"tokens_per_second={len(tokenizer.encode(res))/(t_cost)} tks/s")